# Load Libraries

In [ ]:
import os
import glob

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import ndimage, misc
import random
from tqdm.notebook import tqdm
import pydicom # Handle MRI images

import cv2  # OpenCV - https://docs.opencv.org/master/d6/d00/tutorial_py_root.html

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score


import tensorflow as tf
from tensorflow import keras

from matplotlib.animation import FuncAnimation
from IPython.display import IFrame
from IPython.core.display import display, HTML

# Configuration, Constants, Setup

In [ ]:
data_dir = Path('../input/rsna-miccai-brain-tumor-radiogenomic-classification/')

mri_types = ["FLAIR", "T1w", "T2w", "T1wCE"]
excluded_images = [109, 123, 709] # Bad images

# Load Datasets

In [ ]:
train_df = pd.read_csv(data_dir / "train_labels.csv",
#                        index='id',
#                       nrows=100000
                      )
test_df = pd.read_csv(data_dir / "sample_submission.csv")
sample_submission = pd.read_csv(data_dir / "sample_submission.csv")

train_df = train_df[~train_df.BraTS21ID.isin(excluded_images)]

print(f"train data: Rows={train_df.shape[0]}, Columns={train_df.shape[1]}")
# print(f"test data : Rows={test_df.shape[0]}, Columns={test_df.shape[1]}")

In [ ]:
plt.figure(figsize=(5, 5))
plt.title('Train csv')
sns.countplot(data=train_df, x="MGMT_value");

# Utility Functions

In [ ]:
def load_dicom(path, size = 224):
   
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    # transform data into black and white scale / grayscale
#     data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return cv2.resize(data, (size, size))

In [ ]:
def get_all_image_paths(brats21id, image_type, folder='train'): 
   
    assert(image_type in mri_types)
    
    patient_path = os.path.join(
        "../input/rsna-miccai-brain-tumor-radiogenomic-classification/%s/" % folder, 
        str(brats21id).zfill(5),
    )

    paths = sorted(
        glob.glob(os.path.join(patient_path, image_type, "*")), 
        key=lambda x: int(x[:-4].split("-")[-1]),
    )
    
    num_images = len(paths)
    
    start = int(num_images * 0.25)
    end = int(num_images * 0.75)

    interval = 3
    
    if num_images < 10: 
        interval = 1
    
    return np.array(paths[start:end:interval])

def get_all_images(brats21id, image_type, folder='train', size=225):
    return [load_dicom(path, size) for path in get_all_image_paths(brats21id, image_type, folder)]

In [ ]:
def visualize_sample(
    brats21id, 
    slice_i,
    mgmt_value,
    types=("FLAIR", "T1w", "T1wCE", "T2w")
):
    plt.figure(figsize=(16, 5))
    patient_path = os.path.join(
        IMG_PATH_TRAIN, 
        str(brats21id).zfill(5),
    )
    for i, t in enumerate(types, 1):
        t_paths = sorted(
            glob.glob(os.path.join(patient_path, t, "*")), 
            key=lambda x: int(x[:-4].split("-")[-1]),
        )
        data = load_dicom(t_paths[int(len(t_paths) * slice_i)])
        plt.subplot(1, 4, i)
        plt.imshow(data, cmap="gray")
        plt.title(f"{t}", fontsize=16)
        plt.axis("off")

    plt.suptitle(f"MGMT_value: {mgmt_value}", fontsize=16)
    plt.show()

In [ ]:
KAGGLE_DIR = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'
IMG_PATH_TRAIN = KAGGLE_DIR + 'train/'
for i in random.sample(range(train_df.shape[0]), 10): # get 10 random indexes from the train ds
    _brats21id = train_df.iloc[i]["BraTS21ID"] # for these indexes get the associated brats ID
    _mgmt_value = train_df.iloc[i]["MGMT_value"] # and tumor class
    visualize_sample(brats21id=_brats21id, mgmt_value=_mgmt_value, slice_i=0.5) # visualize samples

# 3D Resconstruction from a 2D Image - Visualization

In [ ]:
import re
def natural_sort(li): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(li, key=alphanum_key)

In [ ]:
def get_image_plane(dicomFile):
    '''
    Returns the MRI's plane from the dicom data.
    
    '''
#     print(dicomFile.get("ImageOrientationPatient"))
    x1,y1,_,x2,y2,_ = [round(j) for j in dicomFile.get("ImageOrientationPatient")]
    cords = [x1,y1,x2,y2]

    if cords == [1,0,0,0]:
        return 'coronal'
    if cords == [1,0,0,1]:
        return 'axial'
    if cords == [0,1,0,0]:
        return 'sagittal'

In [ ]:
def get_voxel(patientFolder, scan_type):
    
    dcm_dir = os.path.join(patientFolder, scan_type)
#     dcm_dir = os.path.join(config["train"], study_id, scan_type)
#     dcm_dir = data_root.joinpath(DATASET, study_id, scan_type)

    # This is a naive way of sorting, not based on metadata
#     dcm_paths = sorted(glob.glob(dcm_dir + "/*.dcm"), key=lambda x: int(x.stem.split("-")[-1]))
    dcm_paths = natural_sort(glob.glob(dcm_dir + "/*.dcm"))
    
    imgs = []
    positions = []
    
    size = -1
    for dcm_path in dcm_paths:
        img = pydicom.dcmread(str(dcm_path))
        imgs.append(img.pixel_array)
        if size == -1:
            size = img.pixel_array.shape
        elif size != img.pixel_array.shape:
            print("Inconsistent size of image: " + str(img.pixel_array.shape))

        positions.append(img.ImagePositionPatient)
    
    plane = get_image_plane(img)
    # Notice that all images are supposed to have the same size
    voxel = np.stack(imgs)
    
    # reorder planes if needed and rotate voxel
    # This is also a naive way to determine if the order of images need to be re-ordered.
    if plane == "coronal":
        if positions[0][1] < positions[-1][1]:
            voxel = voxel[::-1]
            print(f"{dcm_dir[-9:]} {scan_type} {plane} reordered")
        voxel = voxel.transpose((1, 0, 2))
    elif plane == "sagittal":
        if positions[0][0] < positions[-1][0]:
            voxel = voxel[::-1]
            print(f"{dcm_dir[-9:]} {scan_type} {plane} reordered")
        voxel = voxel.transpose((1, 2, 0))
        voxel = np.rot90(voxel, 2, axes=(1, 2))
    elif plane == "axial":
        if positions[0][2] > positions[-1][2]:
            voxel = voxel[::-1]
            print(f"{dcm_dir[-9:]} {scan_type} {plane} reordered")
        voxel = np.rot90(voxel, 2)
    else:
        raise ValueError(f"Unknown plane {plane}")
#     return voxel, plane
    return voxel

In [ ]:
def normalize_contrast(voxel):
    if voxel.sum() == 0:
        return voxel
    voxel = voxel - np.min(voxel)
    voxel = voxel / np.max(voxel)
    voxel = (voxel * 255).astype(np.uint8)
    return voxel

In [ ]:
def crop_voxel(voxel):
    if voxel.sum() == 0:
        return voxel
    keep = (voxel.mean(axis=(0, 1)) > 0)
    voxel = voxel[:, :, keep]
    keep = (voxel.mean(axis=(0, 2)) > 0)
    voxel = voxel[:, keep, :]
    keep = (voxel.mean(axis=(1, 2)) > 0)
    voxel = voxel[keep, :, :]
    return voxel

In [ ]:
voxel = get_voxel("/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00688", "T2w")

In [ ]:
print(type(voxel))
print(voxel.mean())
print(voxel.shape)

print("\nNow crop this voxel \n")
voxel = crop_voxel(voxel)
print(voxel.mean())
print(voxel.shape)

print("\nNow Downsample this voxel \n")
x = voxel.shape[0]
y = voxel.shape[1]
z = voxel.shape[2]

downsampled_voxel = ndimage.zoom(voxel, (32/x, 24/y, 34/z))
print(downsampled_voxel.mean())
print(downsampled_voxel.shape)

In [ ]:
z,x,y = downsampled_voxel.nonzero()
fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

ax.scatter(x, y, z)
plt.show()

# Load Images 

In [ ]:
def get_all_data_for_train(image_type, image_size=32):
    global train_df
    
    X = []
    y = []
    train_ids = []

    for i in tqdm(train_df.index):
        x = train_df.loc[i]
        images = get_all_images(int(x['BraTS21ID']), image_type, 'train', image_size)
        label = x['MGMT_value']

        X += images
        y += [label] * len(images)
        train_ids += [int(x['BraTS21ID'])] * len(images)
        assert(len(X) == len(y))
    return np.array(X), np.array(y), np.array(train_ids)

In [ ]:
def get_all_data_for_test(image_type, image_size=32):
    global test_df
    
    X = []
    test_ids = []

    for i in tqdm(test_df.index):
        x = test_df.loc[i]
        images = get_all_images(int(x['BraTS21ID']), image_type, 'test', image_size)
        X += images
        test_ids += [int(x['BraTS21ID'])] * len(images)

    return np.array(X), np.array(test_ids)

In [ ]:
X, y, trainidt = get_all_data_for_train('T1wCE', image_size=32)
X_test, testidt = get_all_data_for_test('T1wCE', image_size=32)

In [ ]:
X.shape, y.shape, trainidt.shape

# Train/Validation Split

In [ ]:
X_train, X_valid, y_train, y_valid, trainidt_train, trainidt_valid = train_test_split(X, y, trainidt, test_size=0.2, random_state=42)

## Adding a Dimension

In [ ]:
X_train.shape

In [ ]:
X_train = tf.expand_dims(X_train, axis=-1)
X_valid = tf.expand_dims(X_valid, axis=-1)

In [ ]:
X_train.shape

## One-hot encode labels

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)

# Tensorflow Models

## MODEL 1 - ReLU ACTIVATION FUNCTION

In [ ]:
def get_model1():
    np.random.seed(0)
    random.seed(12)
    tf.random.set_seed(12)

    inpt = keras.Input(shape=X_train.shape[1:])

    h = keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(inpt)
    

    
    h = keras.layers.Conv2D(64, kernel_size=(4, 4), activation="relu", name="Conv_1")(h)
    h = keras.layers.MaxPool2D(pool_size=(2, 2))(h)

    h = keras.layers.Conv2D(32, kernel_size=(2, 2), activation="relu", name="Conv_2")(h)
    h = keras.layers.MaxPool2D(pool_size=(1, 1))(h)

    h = keras.layers.Dropout(0.1)(h)

    h = keras.layers.Flatten()(h)
    h = keras.layers.Dense(32, activation="relu")(h)
    h = keras.layers.Dense(16, activation="relu")(h)

    output = keras.layers.Dense(2, activation="softmax")(h)

    model = keras.Model(inpt, output)

    roc_auc = tf.keras.metrics.AUC(name='roc_auc', curve='ROC')

    model.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=[roc_auc,'accuracy']
    )
    return model

## Model 2 - SIGMOID ACTIVATION FUNCTION



In [ ]:
def get_model2():
    np.random.seed(0)
    random.seed(12)
    tf.random.set_seed(12)

    inpt = keras.Input(shape=X_train.shape[1:])

    h = keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(inpt)
    

    
    h = keras.layers.Conv2D(64, kernel_size=(4, 4), activation="sigmoid", name="Conv_1")(h)
    h = keras.layers.MaxPool2D(pool_size=(2, 2))(h)

    h = keras.layers.Conv2D(32, kernel_size=(2, 2), activation="sigmoid", name="Conv_2")(h)
    h = keras.layers.MaxPool2D(pool_size=(1, 1))(h)

    h = keras.layers.Dropout(0.1)(h)

    h = keras.layers.Flatten()(h)
    h = keras.layers.Dense(32, activation="sigmoid")(h)
    h = keras.layers.Dense(16, activation="sigmoid")(h)

    output = keras.layers.Dense(2, activation="softmax")(h)

    model = keras.Model(inpt, output)

    roc_auc = tf.keras.metrics.AUC(name='roc_auc', curve='ROC')

    model.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=[roc_auc,'accuracy']
    )
    return model

## Model 3 - tanh ACTIVATION FUNCTION

In [ ]:
def get_model3():
    np.random.seed(0)
    random.seed(12)
    tf.random.set_seed(12)

    inpt = keras.Input(shape=X_train.shape[1:])

    h = keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(inpt)
    

    
    h = keras.layers.Conv2D(64, kernel_size=(4, 4), activation="tanh", name="Conv_1")(h)
    h = keras.layers.MaxPool2D(pool_size=(2, 2))(h)

    h = keras.layers.Conv2D(32, kernel_size=(2, 2), activation="tanh", name="Conv_2")(h)
    h = keras.layers.MaxPool2D(pool_size=(1, 1))(h)

    h = keras.layers.Dropout(0.1)(h)

    h = keras.layers.Flatten()(h)
    h = keras.layers.Dense(32, activation="tanh")(h)
    h = keras.layers.Dense(16, activation="tanh")(h)

    output = keras.layers.Dense(2, activation="softmax")(h)

    model = keras.Model(inpt, output)

    roc_auc = tf.keras.metrics.AUC(name='roc_auc', curve='ROC')

    model.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=[roc_auc,'accuracy']
    )
    return model

## Set up Model Checkpoint

In [ ]:
checkpoint_filepath = "best_model.h5"

model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor="val_roc_auc",
    mode="max",
    save_best_only=True,
    save_freq="epoch",
    verbose=1,
)

In [ ]:
# # early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor=tf.keras.metrics.AUC(), mode='auto', verbose=1, patience=5)
# # early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)
# early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor="val_roc_auc", mode='max', patience=3)

In [ ]:
# model = get_model02() # LB score 0.676
model1 = get_model1() # LB score 0.5
model1.summary()


In [ ]:
# history = model.fit(x=X_train, y = y_train, epochs=20, 
#                     callbacks=[model_checkpoint_cb], 
#                     validation_data=(X_valid, y_valid))
import matplotlib.pyplot as plt
history = model1.fit(x=X_train, y = y_train, epochs=100, 
                    validation_data=(X_valid, y_valid))


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy - ReLU')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss - ReLU')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model2 = get_model2() # LB score 0.5
model2.summary()

In [ ]:
history = model2.fit(x=X_train, y = y_train, epochs=100, 
                    validation_data=(X_valid, y_valid))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy - Sigmoid')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss - Sigmoid')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model3 = get_model3() # LB score 0.5
model3.summary()

In [ ]:
history = model3.fit(x=X_train, y = y_train, epochs=100, 
                    validation_data=(X_valid, y_valid))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy - tanh')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss - tanh')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Predictions on TEST DATASET

In [ ]:
y_pred = model1.predict(X_valid)

pred = np.argmax(y_pred, axis=1)

result = pd.DataFrame(trainidt_valid)
result[1] = pred

result.columns = ["BraTS21ID", "MGMT_value"]
result2 = result.groupby("BraTS21ID", as_index=False).mean()

result2 = result2.merge(train_df, on="BraTS21ID")
auc = roc_auc_score(
    result2.MGMT_value_y,
    result2.MGMT_value_x,
)

print(f"Test AUC={auc}")

In [ ]:
result2.MGMT_value_y

In [ ]:
result2.MGMT_value_x